In [ ]:
import json
import os
import pathlib
import time
import geopandas as gpd
import pandas as pd

import requests
from requests.auth import HTTPBasicAuth
from planet import Session, DataClient, OrdersClient, Auth, Planet

crs = "EPSG:4326"
sb_bbox = [-125, 34.25, -119.0, 38.0]

data_api_url = "https://api.planet.com/data/v1"
orders_api_url = 'https://api.planet.com/compute/ops/orders/v2' 



Set api key and authorize

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
# API key from Joshua's repo:
if os.environ.get('PL_API_KEY') == None:
    os.environ["PL_API_KEY"] = 'PLAK723f32b266fa484cb256071a3006e80b'
    
planet_key = os.environ.get("PL_API_KEY")

# authentication
auth = HTTPBasicAuth(planet_key, "")
response = requests.get(data_api_url, auth=auth)
print(response)

# make function for pagenation
def p(data):
    print(json.dumps(data, indent = 2))

init session

In [ ]:
# setup
session = requests.Session()

# authenticate
session.auth = (planet_key, "")

res = session.get(data_api_url)

test polygon

In [ ]:
! dir

In [ ]:
sb_county = gpd.read_file("greater_UCSB-campus-aoi.geojson")
type(sb_county)

get geojson

In [ ]:
sb_json = (
    dict( # convert to dict because first index outputs as list
        sb_county.pipe(gpd.GeoDataFrame)
        .to_geo_dict() # convert polygon to geojson format
        ["features"][0] # select first and only feature
        )
        ["geometry"] # select geometry attribute
    )

len(sb_json["coordinates"][0])

In [ ]:
# output the AOI
sb_county.simplify(0.03).to_frame(name="geometry").pipe(gpd.GeoDataFrame).boundary.iloc[0]

Set filters

In [ ]:
# set geometry filter
geometry_filter = {
    "type": "GeometryFilter", 
    "field_name": "geometry", 
    "config": sb_json
}

# # set date filter
# date_range_filter = {
#     "type": "DateRangeFilter", 
#     "field_name": "acquired",  
#     "config": { 
#         "gte": "2023-12-01T00:00:00.000Z",
#         "lt":  "2024-01-01T00:00:00.000Z"
#     }
# }

# DEMO DATE RANGE FILTER
# ONLY FOR ONE MONTH
date_range_filter = {
    "type": "DateRangeFilter", 
    "field_name": "acquired",  
    "config": { 
        "gte": "2022-05-01T00:00:00.000Z",
        "lt":  "2022-06-01T00:00:00.000Z"
    }
}

# # set cloud cover filter
# cloud_cover_filter = {
#     "type": "RangeFilter", 
#     "field_name": "cloud_cover", 
#     "config": {
#         "lt": 0.5
#     }
# }

# MAKING CLOUD FILTER STRICTER TO MAKE NUM SCENES UNDER 500
cloud_cover_filter = {
    "type": "RangeFilter", 
    "field_name": "cloud_cover", 
    "config": {
        "lt": 0.01
    }
}

# combine filters
combined_filters = {
    "type": "AndFilter", 
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

Run quick search based on filters

In [ ]:
item_types = ["PSScene"]

search_request = {
    "item_types": item_types,
    "filter": combined_filters
}

search_result = \
    requests.post(
        "https://api.planet.com/data/v1/quick-search",
        auth = HTTPBasicAuth(planet_key, ''), 
        json = search_request
    )

print(search_result)

Print results scene ids

In [ ]:
first_ids = [feature['id'] for feature in search_result.json()["features"]]
first_ids

Print all scene ids

In [ ]:
all_ids = []
all_ids.extend(first_ids)
loop_trigger = len(first_ids)

id_json = search_result.json()

while loop_trigger == 250:
    next_url = id_json["_links"]["_next"]

    next_250 = session.get(next_url)

    id_json = next_250.json()

    features = next_250.json()["features"]


    id_list = []

    for f in features:
        
        # print id for each feature
        id_str = f["id"]
        id_list.extend([id_str])

    all_ids.extend(id_list)

    print(f"num ids: {len(all_ids)}")

    loop_trigger = len(id_list)

In [ ]:
# not_str_cnt = 0
# for id in all_ids:
#     if isinstance(id, str) != True:
#         print(f"{id} not string!!")
#         not_str_cnt = not_str_cnt + 1

# print(f"not string count: {not_str_cnt}")

In [ ]:
all_ids

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/apis/orders/reference/#tag/Orders/operation/listOrders) API reference.

In [ ]:
auth = HTTPBasicAuth(planet_key, "")
order_response = requests.get(orders_api_url, auth=auth)
print(order_response)

Print previous orders

In [ ]:
orders = order_response.json()['orders']
[r['name'] for r in orders[:5]]

Make order request

In [ ]:
# set content type to json
headers = {"content-type": "application/json"}

# init order parameters
product = [
    { 
        "item_ids": all_ids, 
        "item_type": "PSScene", 
        "product_bundle": "analytic_8b_sr_udm2", 
    }
]

order_request = {
    "name": "wildfire_imagery", 
    "products": product, 
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
def place_order(request, auth):

    # make order request
    response = requests.post(
        orders_api_url, 
        data = json.dumps(request), 
        auth = auth, 
        headers = headers
        )
    print(response.json())

    # get ids of scenes
    order_id = response.json()["id"]
    print(order_id)

    # construct the url of our order
    order_url = orders_api_url + '/' + order_id
    
    return order_url

In [ ]:
# requests.post(
#         orders_api_url, 
#         data = json.dumps(clip_request), 
#         auth = auth, 
#         headers = headers
#         ).json()

In [ ]:
#print(json.dumps(clip_request, indent = 4))

In [ ]:
# place our order
#order_url = place_order(order_request, auth)

### Poll for order success

In [ ]:
def poll_for_success(order_url, auth, num_loops = 30): 
    i = 0
    while(i < num_loops): 

        # iterate
        i += 1

        # get order request
        r = requests.get(order_url, auth = auth)
        response = r.json()

        # grab current state
        state = response["orders"][0]["state"]
        print(state)

        # compare it to a variety of end states and print it
        end_states = ["success", "failed", "partial"]
        if state in end_states:
            print(f"End State: {state}")
            break

        # wait 30 secs
        time.sleep(30)

poll_for_success(orders_api_url, auth)

In [ ]:
# test = requests.get(orders_api_url, auth = auth).json()
# test["orders"][0]["state"]

### Clip to AOI: sb_json    

In [ ]:
# init clip parameters
clip =  {
    "clip": {
        "aoi": sb_json
    }
}

clip_request = { 
    "name": "wildfire santa barbara", 
    "products": product, 
    "tools": [clip], 
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

##### Place order and check for order success

In [ ]:
# we don't have permission to run the clip tool
clip_order_url = place_order(clip_request, auth)

In [ ]:
# This isn't a clip error. 
poll_for_success(clip_order_url, auth)

View results of order

In [ ]:
r = requests.get(orders_api_url, auth = auth)
clip_response = r.json()
order_results = clip_response["orders"][0]["state"]
order_results

### download

In [ ]:
# download results locally and individually
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('..', '..', '..', 'data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [ ]:
download_results(order_results)